# 資料分類

## 背景與動機
* 這種整數鍵值,稱為分類或是字典編碼 
* pandas:[營養補充站](https://byuidatascience.github.io/python4ds/factors.html)

In [ ]:
import numpy as np;import pandas as pd

In [ ]:
values = pd.Series(['apple','orange','apple','apple']*2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [ ]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [ ]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [ ]:
values = pd.Series([0,1,0,0]*2)
dim = pd.Series(['apple','orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [ ]:
dim

0     apple
1    orange
dtype: object

In [ ]:
dim.take(values)#將原有的字串還原

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

##　pandas中的分類型態

In [ ]:
fruits = ['apple','orange','apple','apple']*2
N = len(fruits)
df = pd.DataFrame({'fruit':fruits,
           'basket_id':np.arange(N),
           'count':np.random.randint(3,15,size=N),
           'weight':np.random.uniform(0,4,size=N)},
           columns=['basket_id','fruit','count','weight'])
df

,basket_id,fruit,count,weight
0,0,apple,8,2.710391
1,1,orange,10,1.472005
2,2,apple,13,2.198382
3,3,apple,12,1.950441
4,4,apple,13,2.649784
5,5,orange,7,1.119901
6,6,apple,5,2.966445
7,7,apple,8,3.509737


In [ ]:
fruit_cat = df['fruit'].astype('category')#轉成分類
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [ ]:
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

In [ ]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [ ]:
c.codes#屬性

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [ ]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [ ]:
my_categories =pd.Categorical(['foo','bar','baz','foo','bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [ ]:
#另一個from_codes的建構子
categories = ['foo','bar','baz']
codes = [0,1,2,0,0,1]
my_cats_2 = pd.Categorical.from_codes(codes,categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [ ]:
#可是這樣只有類別沒有順序概念order
ordered_cat = pd.Categorical.from_codes(codes,categories,ordered=True)#categories是level關鍵
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [ ]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

## 分類表示的計算

In [ ]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [ ]:
bins = pd.qcut(draws,4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] <
                                           (0.63, 3.928]]

In [ ]:
bins = pd.qcut(draws,4,labels=['Q1','Q2','Q3','Q4'])
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [ ]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [ ]:
bins = pd.Series(bins,name='quartile')
results = (pd.Series(draws)
        .groupby(bins)
        .agg(['count','min','max'])
        .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [ ]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

## 用分類提升效能

In [ ]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo','bar','baz','qux']) * (N//4)
categories = labels.astype('category')
labels.memory_usage()#??記憶體

160

In [ ]:
categories.memory_usage()

336

## 分類方法

* pandas 中Series的分類方法

| 🦐方法 | 😆描述 |
| ------ |:------:|
|add_categories|在現有類別的尾端加入新的(未使用)類別|
|as_ordered|讓類別有順序|
|as_unordered|讓類別無順序|
|remove_categories|移除類別,將移除掉的值設為null|
|remove_unused_categories|移除在類別資料中未出現的類別|
|rename_categories|用指定新類別名稱集合取代現有類別:但不能改變類別數量|
|reorder_categories|與rename_categories類似,但同時將分類順序|
|set_categories|用指定新分類名稱集合取代現有類別:可新增或移除分類項目|

In [ ]:
s = pd.Series(['a','b','c','d']*2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [ ]:
#特殊屬性cat讓我們可以存取那些分類方法
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [ ]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [ ]:
actual_categories = ['a','b','c','d','e']
cat_s2 = cat_s.cat.set_categories(actual_categories)#如果比實際的觀測值4個還多的時候可以使用set_categories
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [ ]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
dtype: int64

In [ ]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
dtype: int64

In [ ]:
cat_s3 = cat_s[cat_s.isin(['a','b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [ ]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

In [ ]:
cat_s3#remove_unused_categories()可以只留a b類別

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

## 為建模建立指標變數dummy variables

In [ ]:
cat_s = pd.Series(['a','b','c','d'] * 2,dtype='category')

In [ ]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [ ]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


# 進階的Groupby用法

## 分組轉換以及未包裝分組
* 他可以產生一個常數,用於廣播到所有分組
* 他可以產生跟分組一樣大小的一個物件
* 他的輸入不能是mutate類型

In [ ]:
df = pd.DataFrame({'key':['a','b','c'] * 4,
           'value':np.arange(12)})
df

,key,value
0,a,0
1,b,1
2,c,2
3,a,3
4,b,4
5,c,5
6,a,6
7,b,7
8,c,8
9,a,9


In [ ]:
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [ ]:
#我們想改為產生一個df['value']相同大小的Series,但值被換成已'key'分組的平均值的話
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [ ]:
#換個作法
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [ ]:
g.transform(lambda x: x * 2)#就只是做兩倍

0      0
1      2
2      4
3      6
4      8
5     10
6     12
7     14
8     16
9     18
10    20
11    22
Name: value, dtype: int64

In [ ]:
g.transform(lambda x:x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [ ]:
# 正規化的fuction
def normalize(x):
  return (x - x.mean())/x.std()

In [ ]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [ ]:
# 同理
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [ ]:
g.transform('mean')#使用聚合的內建,如mean或是sum都很快

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [ ]:
normalized = (df['value']- g.transform('mean')/g.transform('std'))
normalized#還是一樣

0    -1.161895
1    -0.420094
2     0.321707
3     1.838105
4     2.579906
5     3.321707
6     4.838105
7     5.579906
8     6.321707
9     7.838105
10    8.579906
11    9.321707
Name: value, dtype: float64

## 分組時間重新取樣

In [ ]:
N = 15
times =  pd.date_range('2017-05-20 00:00',freq='1min',periods=N)
df = pd.DataFrame({'time':times,'value':np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [ ]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [ ]:
df2 = pd.DataFrame({'time':times.repeat(3),
           'key':np.tile(['a','b','c'],N),
           'value':np.arange(N*3.)})
df[:7]

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6


In [ ]:
#https://github.com/wesm/pydata-book/issues/121
#time_key = pd.TimeGrouper('5min')
#resampled = (df.set_index('time').groupby(['key',time_key].sum()))
#resampled
#resampled.reset_index()

# 方法串接的技巧
```python
df = load_data()
df2 = df2[df['col2']<0]
df2['col1_demeaned'] = df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
```
另一種寫法
```python
#不使用函式的做法
df2=df.copy
df2['k']=v
#使用函式作法
df2=df.assign(k=v)

result =(df.assign(col_demeaned=df2.col1-df.col2.mean()).groupby('key').col1_demeaned.std())
```
展示callable
```python
df = load_data()
df2 = df[df['col2']<0]
```
可改成
```python
df(load_data()
    [lambda x:x['col2']<0])
```
成果方法集成
```python
result=(load_data()
      [lambda x:x.col2<0]
      .assign(col1_demeaned=lambda x:x.col1-x.col1.mean())
      .groupby('key')
      .col1_demeaned.std())

```

## pip方法
假設有一串函式:
```python
a = f(df.arg1=v1)
b = g(a,v2,arg3=v3)
c = h(b,arg4=v4)
```
當使用函式可以接受Series或DataFrame物件當作輸入和輸出時,你就可以將上述的程式改寫成呼叫pipe
```python
result= (df.pipe(f,arg1=v1)
      .pipe(g,v1,arg3=v3)
      .pipe(h,arg4=v4))
```

另一個範例

```python
g = df.groupby(['key1','key2'])
df['col1']=df['col1']-s.tranform('mean')

def group_demean(df,by,cols):
  result = df.copy()
  g = df.groupby(by)
  for c in cols:
    result[c]=df[c]-g[c].transform('mean')
  return result

result = (df[df.col1<0].pipe(group_demean,['key1','key2'],['col1']))
```
